Scraping the Data and storing the raw data

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS layers;
CREATE SCHEMA IF NOT EXISTS layers.bronze;
-- CREATE VOLUME IF NOT EXISTS layers.bronze.book_daily_raw_volume ;
CREATE SCHEMA IF NOT EXISTS layers.silver;
CREATE SCHEMA IF NOT EXISTS layers.gold;

In [0]:
spark.read.format("delta").table("layers.silver.book_master").display()

In [0]:
%sql

CREATE TABLE IF NOT EXISTS layers.bronze.book_daily_raw (
  book_name STRING,
  isbn STRING,
  author STRING,
  source STRING,
  price DOUBLE,
  scrape_ts TIMESTAMP,
  url STRING
)
USING DELTA
PARTITIONED BY (scrape_ts)

In [0]:
dbutils.widgets.text("csv_file", "/Workspace/Users/jennoronha28@gmail.com/Project/files/daily.csv", "CSV File Path")
csv_file_path = dbutils.widgets.get("csv_file")

from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import to_timestamp, col, from_utc_timestamp

# Define schema first
schema = StructType([
    StructField("isbn", StringType(), True),
    StructField("book_name", StringType(), True),
    StructField("author", StringType(), True),
    StructField("source", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("scrape_ts", StringType(), True),
    StructField("url", StringType(), True)
])

sample_data = spark.read.csv(csv_file_path, schema=schema, header=True)
df_sample = sample_data.withColumn("scrape_ts", from_utc_timestamp(to_timestamp(col("scrape_ts")), "Asia/Kolkata"))
display(df_sample)


In [0]:
df_sample.printSchema()

In [0]:
%python
# Register temp view
df_sample.createOrReplaceTempView("temp_view_today")

In [0]:
%sql

MERGE INTO layers.bronze.book_daily_raw t
USING temp_view_today s
ON t.source = s.source
AND t.scrape_ts = s.scrape_ts
-- AND t.scrape_date = s.scrape_date
WHEN NOT MATCHED THEN
INSERT (
  book_name,
  author,
  source,
  price,
  scrape_ts,
  isbn,
  url
)
VALUES (
  s.book_name,
  s.author,
  s.source,
  s.price,
  s.scrape_ts,
  s.isbn,
  s.url
);

SELECT * FROM layers.bronze.book_daily_raw;

In [0]:
from delta.tables import DeltaTable
delta_table = DeltaTable.forName(spark, "layers.bronze.book_daily_raw")

# Show Delta table version history
history_df = delta_table.history()
display(history_df)

In [0]:
%sql
DESCRIBE DETAIL layers.bronze.book_daily_raw

In [0]:
%sql
select * FROM layers.bronze.book_daily_raw